In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import h5py
import pandas as pd
from transformers import T5EncoderModel, T5Tokenizer
from transformers import BertForMaskedLM, BertTokenizer, pipeline
from Bio import SeqIO

def get_t5_embeddings(input_fasta, output_embeddings_h5, output_embeddings_csv, model_dir=None):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Using device: {}".format(device))

    # Load T5 model and tokenizer(prot_t5_xl_bfd, prot_bert, prot_t5_xl_uniref50, prot_albert,prot_xlnet)
    transformer_link = "Rostlab/prot_t5_xl_uniref50"
    # tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False)
    # model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert_bfd")
    model = T5EncoderModel.from_pretrained(transformer_link, cache_dir=model_dir)
    model = model.to(device)
    model = model.eval()
    vocab = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False)
    # Read protein sequences from FASTA file
    sequences = read_fasta(input_fasta)

    # Process sequences and obtain embeddings
    embeddings = {}
    for identifier, sequence in zip(sequences["ID"], sequences["Seq"]):
        sequence = sequence.replace('U', 'X').replace('Z', 'X').replace('O', 'X')
        sequence = ' '.join(list(sequence))

        token_encoding = vocab.encode_plus(sequence, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(token_encoding['input_ids']).to(device)
        attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

        with torch.no_grad():
            embedding_repr = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))

        # Slice-off padded/special tokens
        emb = embedding_repr.last_hidden_state.squeeze(0)

        embeddings[identifier] = emb.mean(dim=0).detach().cpu().numpy().squeeze()

    # Save embeddings to an HDF5 file
    with h5py.File(output_embeddings_h5, "w") as hf:
        for sequence_id, embedding in embeddings.items():
            hf.create_dataset(sequence_id, data=embedding)

    # Save embeddings to a CSV file
    df = pd.DataFrame.from_dict(embeddings, orient='index')
    df.to_csv(output_embeddings_csv)

    print('Total number of embeddings: {}'.format(len(embeddings)))
    return embeddings




Current Working Directory: /content
Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Read fasta file DONE!
